# Category Life List

This notebook generates and exports a life list for all sightings of the species in a category. For each species in that category, the total number of sightings of the species and the number of locations where sightings were recorded are shown alongside the species name. To use it, update the category in the first code cell, below, before running the notebook.

In [ ]:
# Species category to report on
category = ""

In [ ]:
from pathlib import Path
import sqlparse

# Read the query file
query_file_path = Path("sql") / "category_life_list.sql"
with open(query_file_path.absolute(), "r") as f:
    query = f.read().replace("\n", " ")

# Replace the placeholders
query = query.replace("$CATEGORY", category)

# Show a pretty-printed form of the query
print(sqlparse.format(query, reindent=True, keyword_case='upper'))

In [ ]:
import pandas as pd
import sqlite3
import os

# Connect to the database, execute the query and read the results into a dataframe
database_path = os.environ["NATURE_RECORDER_DB"]
connection = sqlite3.connect(database_path)
df = pd.read_sql_query(query, connection, parse_dates=["Date"])

# Check there is some data
if not df.shape[0]:
    message = f"No data found for category '{category}'"
    raise ValueError(message)

In [ ]:
# Aggregate the data to produce the life list
life_list = (
    df
    .groupby("Species")
    .agg(
        Sightings=("Count", "sum"),
        Locations=("Location", pd.Series.nunique)
    )
    .reset_index()
    .sort_values(by="Species", ascending=True)
)


# Print the life list
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    display(life_list)

In [ ]:
import re

# Create the folder to hold exported reports
export_folder_path = Path("exported")
export_folder_path.mkdir(parents=True, exist_ok=True)

# Export the life list
clean_category = re.sub("[^0-9a-zA-Z ]+", "", category).replace(" ", "-")
export_file_path = export_folder_path / f"{clean_category}-Life-List.xlsx"
life_list.to_excel(export_file_path.absolute(), sheet_name="Life List", index=False)